```bash
mosesdecoder/scripts/tokenizer/tokenizer.perl -l en < data/test_example/Europarl_train_10k.en > data/test_example/Europarl_train_10k.tok.en
mosesdecoder/scripts/tokenizer/tokenizer.perl -l fr < data/test_example/Europarl_train_10k.fr > data/test_example/Europarl_train_10k.tok.fr

mosesdecoder/scripts/recaser/train-truecaser.perl --model data/test_example/truecase-model.en --corpus data/test_example/Europarl_train_10k.tok.en
mosesdecoder/scripts/recaser/train-truecaser.perl --model data/test_example/truecase-model.fr --corpus data/test_example/Europarl_train_10k.tok.fr

mosesdecoder/scripts/recaser/truecase.perl --model data/test_example/truecase-model.en < data/test_example/Europarl_train_10k.tok.en > data/test_example/Europarl_train_10k.tok.true.en
mosesdecoder/scripts/recaser/truecase.perl --model data/test_example/truecase-model.fr < data/test_example/Europarl_train_10k.tok.fr > data/test_example/Europarl_train_10k.tok.true.fr

mosesdecoder/scripts/training/clean-corpus-n.perl data/test_example/Europarl_train_10k.tok.true fr en data/test_example/Europarl_train_10k.tok.true.clean 1 80
```

In [3]:
!onmt_build_vocab -config test_example.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-05-29 04:27:33,456 INFO] Counter vocab from 10000 samples.
[2024-05-29 04:27:33,456 INFO] Build vocab on 10000 transformed examples/corpus.
[2024-05-29 04:27:37,255 INFO] Counters src: 11547
[2024-05-29 04:27:37,255 INFO] Counters tgt: 14694


In [6]:
!onmt_train -config test_example.yaml

[2024-05-29 04:38:43,239 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2024-05-29 04:38:43,239 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-05-29 04:38:43,240 INFO] Missing transforms field for valid data, set to default: [].
[2024-05-29 04:38:43,240 INFO] Parsed 2 corpora from -data.
[2024-05-29 04:38:43,240 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-05-29 04:38:43,282 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', 'the', ',', '.', 'of', 'to', 'and']
[2024-05-29 04:38:43,283 INFO] The decoder start token is: <s>
[2024-05-29 04:38:43,283 INFO] Building model...
[2024-05-29 04:38:43,421 INFO] Switching model to float32 for amp/apex_amp
[2024-05-29 04:38:43,421 INFO] Non quantized layer compute is fp32
[2024-05-29 04:38:43,611 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise

In [7]:
!onmt_translate -model ../data/test_example/run/model_step_1000.pt -src ../data/test_example/Europarl_test_500.en -output ../data/test_example/pred_1000.txt -gpu 0 -verbose

Time w/o python interpreter load/terminate:  2.2834973335266113


[2024-05-29 04:43:36,054 INFO] Loading checkpoint from ../data/test_example/run/model_step_1000.pt
[2024-05-29 04:43:36,195 INFO] Loading data into the model
[2024-05-29 04:43:37,907 INFO] 
SENT 1: ['It', 'is', 'only', 'fair', 'to', 'acknowledge', 'the', 'extraordinary', 'contribution', 'of', 'this', '<unk>']
PRED 1: en ce qui concerne l&apos; Union européenne , il n&apos; y a pas le cas .
PRED SCORE: -1.3380

[2024-05-29 04:43:37,908 INFO] 
SENT 2: ['<unk>', '<unk>', 'which', 'has', 'fought', 'to', 'maintain', 'the', '<unk>', 'sense', 'and', 'the', 'balance', 'of', 'the', '<unk>', 'the', 'support', 'of', 'this', 'House', 'has', 'been', 'encouraging', 'and', '<unk>']
PRED 2: en ce qui concerne l&apos; Union européenne , il n&apos; y a pas le fait que l&apos; Union européenne n&apos; a pas le cas .
PRED SCORE: -1.4348

[2024-05-29 04:43:37,908 INFO] 
SENT 3: ['<unk>', 'the', '<unk>', 'Europe', 'has', 'kept', 'moving', '<unk>']
PRED 3: le Parlement n&apos; a pas le cas .
PRED SCORE: -1.2

In [9]:
import nltk

# read the translation result
with open("../data/test_example/pred_1000.txt", "r", encoding="utf-8") as pred_file:
    translations = pred_file.readlines()

# read the corresponding reference file
with open("../data/test_example/Europarl_test_500.fr", "r", encoding="utf-8") as ref_file:
    references = ref_file.readlines()

references = [ref.strip().split() for ref in references]

translations = [trans.strip().split() for trans in translations]

# calculate BLEU score
bleu_score = nltk.translate.bleu_score.corpus_bleu(references, translations, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1)

print("BLEU Score:", bleu_score)


BLEU Score: 0.00023233963151202632
